# ML101 

In this workshop, we will learn about machine learning from first principles. Everything you need for this workshop is contained in this notebook. To test everything is working, execute the import statements below.

In [ ]:
from __future__ import annotations

import sys
from typing import Protocol

import numpy as np
import polars as pl
import requests
from loguru import logger
from sklearn.neural_network import MLPRegressor

logger.remove(0)
logger.add(sys.stderr, format="{message}")

Now we know your environment is set up correctly, we can begin!

## Exemplar dataset

In this workshop, we will use a dataset called [Concrete Compressive Strength](https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength). This workshop includes a couple of files, `train.parquet` and `test.parquet`, where the data has already been prepared for you.


In this workshop, we use the [Polars library](https://pola.rs/) to open and analyse the data. We read in the data like so:

In [ ]:
train_df = pl.read_parquet("./train.parquet")
test_df = pl.read_parquet("./test.parquet")

The `df` in `train_df` and `test_df` is short for DataFrame. A DataFrame is a Python object that represents the underlying tabular data. We can explore the data like so:

In [ ]:
train_df

The DataFrame is composed of nine columns. The first seven columns represent the quanity of material per kg in a cubic metre mixture. The eight column is the number of days since the concrete was mixed. The final column is the compressive strength of the resulting concrete in MPa.

## Our goal

We want to make the best concrete possible. We don't want to make lots of batches of concrete to discover what the optimal mixture should be. Instead, we want to use the existing data to come up with a *model* of the underlying relationship between the possible inputs, and the resulting strength. 

The general problem is not limited to mixing concrete. There are many systems where we want to model an input/output based on some input/output observations. The concrete dataset was selected because:

- It's relatively small (9 columns and around 1000 rows)
- It's nonlinear (more on that later)
- It doesn't contain any categorical variables (i.e., everything is continuous*). This makes the maths much easier while we learn about fundamental machine learning concepts

(*) Age as measured by an integer-valued number of days is "continuous enough" for our purposes.

Let's now look at `test_df`

In [ ]:
test_df

Notice that `test_df` does not contain a "concrete compressive strength" column. `test_df` is a simulation of "deploying to production". We have a number of candidate mixtures, and we need to predict, in advance, the strength of each mixture.

In this workshop, we will create a series of models. We will "deploy" our model to produce a series a predictions on the test dataset. We will send these predictions off to a test server to simulate the quality of our model in production. 

Let's test our connection to the test server. During the workshop, you'll be given an endpoint for the test server. Feel free to choose your own username. If you have the correct endpoint and have selected a *unique* username, you will be provided with a token that you can use to submit your predictions later in the workshop.

In [ ]:
endpoint = "CHANGEME"
username = "CHANGEME"
token = requests.post(f"{endpoint}/users/create/{username}").json()["token"]

## Data exploration

Before we create any models, let's look at some relationships between the input variables and the resulting strength of the concrete.

In [ ]:
train_df.plot.scatter(x="cement", y="concrete compressive strength")

We see that the data are very noisy. If we squint, we can see a general positive relationship between the amount of cement and the resulting strength.

In [ ]:
train_df.plot.scatter(x="water", y="concrete compressive strength")

The relationship between water and strength is less clear. Again, if we squint, we see that low levels of water yield low strength concrete, medium levels of water yield a wide range of strengths, and high levels of water yield low strength concrete again. The transition from low to high to low again indicates a *nonlinear* relationship. We cannot draw a straight line that goes up and down again. We could however imagine drawing something like a "parabola-of-best-fit" through these points. We will talk more about nonlinearity later. For now, know that nonlinearity makes everything more complicated (but also interesting!).

In [ ]:
train_df.plot.scatter(x="age", y="concrete compressive strength")

Age seems to have a positive effect on strength, though the strongest concretes tend to be between 25 and 300 days old. Again, we observe a nonlinear effect where, after some time, age has a negative effect rather than a positive one.

In [ ]:
train_df.plot.scatter(x="superplasticizer", y="concrete compressive strength")

Finally, we observe that superplasticizer has a fairly obvious positive effect, and it appears as though all of the highest-strength concretes contain at least some superplasticizer.

We will break down our training data into two subsets: training and evaluation. The training data is used for the training itself, and the evaluation data is used to locally test how well the model generalises to unseen data. Without having an evaluation dataset, we run the risk of "overfitting" on our training data. An overfitted model will perform poorly against the test data. 

For example, let's say we create a model is trained by hashing the inputs of every row in `train_df` and returns the corrsponding strength. Such a model could be defined as:

In [ ]:
class Model(Protocol):
    def __call__(self, inputs: tuple) -> float: ...


def train_overfit_model() -> Model:
    lookup: dict[tuple, float] = {}
    for record in train_df.iter_rows():
        lookup[record[:-1]] = record[-1]

    return lambda x: lookup[x]

When we evaluate this model on the training data, it will have perfect performance. However, if we submit the model to the test server, we will observe terrible performance (that is, if the model even runs without raising an exception). By contrast, if we had resevered part of the training data as an evaluation dataset that was not considered during training, we could have tested locally and realised how bad the model really was *before* we deployed it to production.

Let's reserve the first 100 records for `eval_df`:

In [ ]:
n_eval = 100
eval_df = train_df[:n_eval]
train_df = train_df[n_eval:]

Let's define a series of helper functions before we start training models.

In [ ]:
def evaluate(model: Model) -> float:
    """Evaluate a given model on `eval_df`."""
    predicted = pl.Series("predicted", [model(x[:-1]) for x in eval_df.iter_rows()])
    return ((eval_df["concrete compressive strength"] - predicted) ** 2).mean()


def generate_test_predictions(model: Model) -> list[float]:
    """Generate model predictions for test_df."""
    return [model(x) for x in test_df.iter_rows()]


def submit_predictions(model_name: str, predictions: list[float]) -> float:
    """Submit predictions for a given model to the test server and receive a score."""
    return requests.post(
        f"{endpoint}/predictions/submit",
        json={"token": token, "model": model_name, "predictions": predictions},
    ).json()["score"]

## Our first model

Let's begin by defining a super simple model that returns a constant for every input. A reasonable constant to return is the mean strength from the training data.

In [ ]:
def train_mean_model() -> Model:
    mean = train_df["concrete compressive strength"].mean()
    return lambda _: mean

Let's "train" our model and evaluate it on `eval_df`

In [ ]:
first_model = train_mean_model()
eval_score = evaluate(first_model)
logger.info(f"Score on evaluation dataset: {eval_score:.3f}")

We received a score after evaluation. What does this score mean?

## Objective functions

In machine learning (and optimisation more generally), we use a so-called *objective function* to measure how good our models are.

Consider the following functions:
- A model $f: \mathcal{X} \rightarrow \mathcal{Y}$ that maps each input in input space $x\in\mathcal{X}$ to an output $y\in\mathcal{Y}$.
- The ground truth $g: \mathcal{X} \rightarrow \mathcal{Y}$ that maps each input in input space $x\in\mathcal{X}$ to the true $y\in\mathcal{Y}$.
- A distance function $d: \mathcal{Y} \times \mathcal{Y} \rightarrow \mathbb{R}_{\geq 0}$, where $\mathcal{Y}$ is the space of possible outputs, and $\mathbb{R}_{\geq 0}$ is the set of non-negative reals. $d(\hat{y},y)$ measures how far away each prediction $\hat{y}=f(x)$ is from the true value $y=g(x)$.
- The expected input distribution $p: \mathcal{X} \rightarrow [0, 1]$ such that $\int_{x\in\mathcal{X}}p(x) = 1$, which measures how likely we will come across each set of inputs in production. This is typically unknowable, and is usually approximated using the training data distribution.

In machine learning, we typically want to minimise the following so-called objective function:

$$
h(f, g) = \sum_{x \in\mathcal{X}} p(x)d\left(f\left(x\right), g\left(x\right)\right)
$$

During evaluation, we estimate the above using the evaluation dataset $E=[x_1, x_2, \dots]$, which simplifies the above equation to:

$$
\tilde h(f, g) = \frac{1}{|E|}\sum_{x \in E} d\left(f\left(x\right), g\left(x\right)\right)
$$

Being a continous problem, a reasonble distance function is the sum of squares:

$$ d(\hat{y}, y) = (\hat{y} - y)^2$$

In machine learning, there are many different distance functions. Fortunately ours is relatively simple!

TL;DR the score that was returned to us previously was the average squared distance from the predicted strength and the actual strength over all examples in the evaluation dataset!

Let's submit our first model to the test server to see how well it does in production.

In [ ]:
first_predictions = generate_test_predictions(first_model)
test_score = submit_predictions("my first model", first_predictions)
logger.info(f"Score on test dataset: {test_score:.3f}")

Notice that the returned score is different from the score we computed locally for the evaluation dataset.

## Linear models

Let's try something more interesting. Rather than just returning a constant, we will create a linear model:

$$f_\theta(x) = w \cdot x + b $$

where $\theta=(w,b)$, $w \in \mathcal{R}^{|x|}$, $b \in\mathcal{R}$. We often use the letter $\theta$ to encapsulate any free parameters associated with the model. Typically, when we refer to "training", we are referring to the process of improving $\theta$ such that the objective function $h$ is gradually minimised.

The following function accepts a weight vector $w$ and a bias term $b$, and returns a linear model.


In [ ]:
def create_linear_model(weights: np.ndarray, bias: float = 0) -> Model:
    return lambda x: np.dot(np.array(x), weights) + bias

How should we select the weights vector $w$ and bias term $b$ such that the objective function is minimised? One way to do this is via random search. In the following block, we randomly select weight vectors and create new linear models. For now, we leave the bias term set to zero.

In [ ]:
best_score = np.inf
best_iteration = None
best_model = None
for i in range(10):
    weights = 0.02 * np.random.rand(len(train_df.columns) - 1)
    random_linear_model = create_linear_model(weights)
    eval_score = evaluate(random_linear_model)
    if eval_score < best_score:
        best_score = eval_score
        best_iteration = i
        best_model = random_linear_model

    logger.info(
        f"Score on evaluation dataset for random linear model {i}: {eval_score:.3f}",
    )
logger.info(
    f"Best score on evaluation dataset was {best_score:.3f} from iteration {best_iteration}",
)

Depending on how lucky you are, you *may* end up with a better model than the constant model we trained earlier. Regardless, let's upload our best random model to the test server to see how well it performs.

In [ ]:
random_predictions = generate_test_predictions(best_model)
test_score = submit_predictions("my random model", random_predictions)
logger.info(f"Score on test dataset: {test_score:.3f}")

As you may guess, random search is inefficient, and it gets exponentially worse as $|\theta|$ grows.

How else could we find good parameters for our linear model? Fortunately, when our distance function is the sum of squares, we can derive an exact solution using the least squares method. Below, we use NumPy to derive the optimal linear model. Don't worry too much about the maths. Just know that, if the objective function is the sum of squares, and the model is linear, we can efficiently find the optimal solution.

In [ ]:
A = np.c_[
    train_df.drop("concrete compressive strength").to_numpy(),
    np.ones(len(train_df)),
]
optimal_params = np.linalg.lstsq(
    A,
    train_df["concrete compressive strength"],
    rcond=None,
)[0]
weights = optimal_params[:-1]
bias = optimal_params[-1]

optimal_linear_model = create_linear_model(weights, bias)
eval_score = evaluate(optimal_linear_model)
logger.info(f"Score on evaluation dataset: {eval_score:.3f}")

The score for the optimal linear model eclipses anything we have developed so far. Let's submit it to the test server!

In [ ]:
optimal_linear_predictions = generate_test_predictions(optimal_linear_model)
test_score = submit_predictions("my linear model", optimal_linear_predictions)
logger.info(f"Score on test dataset: {test_score:.3f}")

One notable advantage of linear models is that they are inherently "interpretible". That is, we can look at the resulting weights and derive meaning. Let's print out the optimal weights. 

In [ ]:
dict(zip(test_df.columns, weights))

The resulting weights are consistent with our analysis of the training data. The amount of cement is positively correlated with the resulting strength, while adding extra water tends to reduce the strength. We observe that the most significant factor in high-strength concrete is the addition of superplasticizer, which is consistent with our earlier observations.

## Limitations of linear models

Linear models are unable to capture nonlinear effects. In our case, the most important nonlinear effects are:
- The nonlinear relationship between some input variables and the resulting strength. For example, the negative correlation value for water suggests that we shouldn't use water at all. This is obviously incorrect as we can't make concrete without water.
- Non-additive coupling effects between input variables - what if a seemingly unimportant input such as coarse aggregate only has a positive effect when correctly combined with other inputs?

To address these limitations, we will now explore nonlinear models. We will first use prebuilt nonlinear models from [scikit-learn](https://scikit-learn.org/stable/index.html) before training some custom neural networks in [PyTorch](https://pytorch.org/).

## scikit-learn

`scikit-learn` contains a number of predefined machine learning models. Select a relevant nonlinear regression model from [here](https://scikit-learn.org/stable/user_guide.html) and try to implement a model that beats the optimal linear model.

Some algorithms you may want to try:
- [Kernel ridge regression](https://scikit-learn.org/stable/modules/kernel_ridge.html)
- [Support vector machines](https://scikit-learn.org/stable/modules/svm.html#regression)
- [Decision trees](https://scikit-learn.org/stable/modules/tree.html#regression)
- [Neural networks](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#regression)

Below, we provide an example for `MLPRegressor`


In [ ]:
y_col = "concrete compressive strength"
X_train = train_df.drop(y_col).to_numpy()
y_train = train_df[y_col].to_numpy()


def train_mlp_model() -> Model:
    regr = MLPRegressor(random_state=42, max_iter=1000).fit(X_train, y_train)
    return lambda x: regr.predict(np.array(x)[np.newaxis])[0]


mlp_model = train_mlp_model()
eval_score = evaluate(mlp_model)
logger.info(f"Score on evaluation dataset: {eval_score:.3f}")

This looks like a big improvement! Let's submit it to the test server.

In [ ]:
mlp_predictions = generate_test_predictions(mlp_model)
test_score = submit_predictions("my mlp model", mlp_predictions)
logger.info(f"Score on test dataset: {test_score:.3f}")

Now it's your time to try and see how far up the leaderboard you can get!

## Stretch goal: PyTorch

For higher dimesional data, such as images and text, we tend to use deep neural networks. While `scikit-learn` supports basic neural networks via the `MLPRegressor` class, it does not support custom neural architectures or GPU training. For this we use dedicated libraries such as [TensorFlow](https://www.tensorflow.org/) and [PyTorch](https://pytorch.org/).

We may cover neural networks in more depth during the workshop. At the very least, an example PyTorch neural network is provided below that you can use and customise.


In [ ]:
%pip install torch --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import copy

import torch
from torch import nn, optim

batch_size = 10
X_train = torch.tensor(X_train, dtype=torch.float32).reshape(-1, batch_size, 8)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, batch_size, 1)

torch_model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1),
)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(torch_model.parameters(), lr=3e-4)

best_weights = None
best_mse = np.inf


def get_model_for_evaluation(model: nn.Module) -> Model:
    def predict(x: tuple) -> float:
        model.eval()
        return float(model(torch.tensor(x).reshape(-1, 8)))

    return predict


for _epoch in range(100):
    torch_model.train()
    for X, y in zip(X_train, y_train):
        y_pred = torch_model(X)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mse = evaluate(get_model_for_evaluation(torch_model))
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(torch_model.state_dict())

torch_model.load_state_dict(best_weights)

final_nn_model = get_model_for_evaluation(torch_model)

eval_score = evaluate(final_nn_model)
logger.info(f"Score on evaluation dataset: {eval_score:.3f}")

Let's submit our PyTorch model to the test server!

In [ ]:
nn_predictions = generate_test_predictions(final_nn_model)
test_score = submit_predictions("my nn model", nn_predictions)
logger.info(f"Score on test dataset: {test_score:.3f}")